In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.tools import Tool
from langchain.agents import (
    AgentType,
    initialize_agent
)

from pydantic import BaseModel, Field

## Setting the LLM

In [ ]:
chat_llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 0,
    openai_api_key = open("openai_api.txt", "r").read()
)

## Need for Extra Tools

Using agents allows us to give LLMs access to `tools`. These tools present an infinite number of possibilities. With tools, LLMs can search the web, do math, run code, and more.

Althoug LangChains pre-build tools, in may `real-world` projects we'll often find that only so many requirements can be satisfied by existing tools. Thus we must modify existing tools or even build entirely new ones.

## Building Tools

At their core, tools are objects that consume some `input`, typically in the format of a string (text), and `output` some helpful information as a string.

In reality, they are little more than a simple function that we'd find in any code. The only `difference` is that `tools` take input from an LLM and feed their output to an LLM.

Besides the actual function that is called a `Tool` consists of several components:
* **name** (str, required) - the unique name of the tool.
* **description** (str, optional-recommended) - determines the tool usage.
* **return_direct** (bool, default-False) - stopes execution and return Tool Observation
* **args_schema** (Pydantic BaseModel, optional-recommended) - used to provide more information (e.g. , few-shot examples) or validation for expected parameters.

In [ ]:
## Creating the First Tool

search = SerpAPIWrapper(serpapi_api_key=open("serpapi_api.txt").read())

tools = [
    Tool.from_function(
        func = search.run,
        name = "Search",
        description = "useful for when you need to answer questions about current events"
        # coroutine = ... <- you can specify an async method if desired as well
    )
]

In [ ]:
## Creating the Second Tool

llm_math_chain = LLMMathChain.from_llm(llm=chat_llm, verbose=True)

class CalculatorInput(BaseModel):
    question: str = Field()


tools.append(
    Tool.from_function(
        func = llm_math_chain.run,
        name = "Calculator",
        description = "useful for when you need to answer questions about math",
        args_schema = CalculatorInput
        # coroutine= ... <- you can specify an async method if desired as well (the asynchronous version of the function)
    )
)

In [ ]:
## Initializing the Agent

agent = initialize_agent(
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools = tools,
    llm = chat_llm,
    verbose = True
)

In [ ]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")